Trends Identified in Data:

1. Game is played primarily by males ages 20-30.  With males ages 20-24 generating about 50% of sales revenue.

2. Although more males play the game, females appear to spend more than males as per player (as per avg purchase price per gender). Also, players ages <10 and >35 appear to spend more per player than males ages 20-24 (as per avg purchase price per age group)

3. Most popular items are some of the more expensive items available for purchase. Suggesting that price may not affect the decision to purchase as much as the attributes of the items purchased.


In [76]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = 'purchase_data.csv'

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# Display Data Sample
purchase_data.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",$3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,$1.56
2,2,Ithergue48,24,Male,92,Final Critic,$4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,$3.27
4,4,Iskosia90,23,Male,131,Fury,$1.44


In [6]:
# Find and Print Total Number of Players
#--------------------------------------------

#Find the total number of unique players

unique_players = []
unique_players = purchase_data['SN'].unique()

#Calculate Total Number of Players and Display in a Dataframe

total_players = len(unique_players)
total_players_dict = {'TotalPlayers' : [total_players]}
total_players_df = pd.DataFrame(total_players_dict)

total_players_df



,TotalPlayers
0,576


In [48]:
# Purchasing Analysis
#--------------------------

## Find Total number of Unique Items

total_items = len(purchase_data["Item ID"].unique())

## Calculate Total Revenue

total_revenue = purchase_data["Price"].sum()

##Calculate Total Number of Purchases

purchase_qty = len(purchase_data["Purchase ID"])

## Calculate Average Purchase Price

avg_price = total_revenue/purchase_qty
#avg_price = avg_price.round(2)
#avg_price = avg_price.astype(float)

##Display Results in Data Frame

# Place Calculated Values into Dictionaries
purchase_analysis = {"Number of Unique Items" : [total_items],
                     "Average Price" : [avg_price],
                     "Number of Purchases" : [purchase_qty],
                     "Total Revenue" : [total_revenue]}

#Apply Formatting
purchase_stats = pd.options.display.float_format = '${:,.2f}'.format


#Create Dataframe with Results
purchase_stats = pd.DataFrame(purchase_analysis)


purchase_stats


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [49]:
# Gender Demographics
#-----------------------------

#Create Dataframe containing only unique player names

# Create copy of data and drop duplicates from purchase data df

purchase_data_no_dups = pd.DataFrame(purchase_data)

# Drop duplicate values in SN
purchase_data_nodups_df = pd.DataFrame(purchase_data_no_dups.drop_duplicates(subset=["SN"]))


## Calculate Total Number of Players by Gender

gender_demo_df = pd.DataFrame(purchase_data_nodups_df["Gender"].value_counts())
#gender_demo_df

# Display result in data frame
gender_totals_df = gender_demo_df.reset_index().rename(columns={"index" : "Gender", "Gender" :"Gender_Count"})

gender_totals_df 

##Calculate Percentage of Player by Gender

gender_totals_df["Percentage_of_Players"] = gender_totals_df["Gender_Count"] / total_players*100

# Format Datafrmae
gender_totals_df["Percentage_of_Players"] = gender_totals_df["Percentage_of_Players"].map('{:,.2f}%'.format)


gender_totals_df




,Gender,Gender_Count,Percentage_of_Players
0,Male,484,84.03%
1,Female,81,14.06%
2,Other / Non-Disclosed,11,1.91%


In [55]:
# Purchasing Analysis By Gender
#--------------------------------------

#Group Data by Gender

#Calculates total purchases and total purchase value for each gender
gender_groups = purchase_data.groupby(["Gender"]).agg({"Purchase ID":"count", "Price":"sum"})
#gender_groups #outputs dataframe

#Format Data Frame with results
gender_purchases_df = gender_groups.reset_index().rename(columns= {"index":"Gender", "Purchase ID":"Purchase_Count","Price":"Total_Purchase_Value"})
#gender_purchases_df

#Calculate Average purchase value per gender
gender_purchases_df["Average_Purchase_Price"] = gender_purchases_df["Total_Purchase_Value"]/gender_purchases_df["Purchase_Count"]
#gender_purchases_df

#Calculate Average Purchase per Person
gender_sn_groups = purchase_data.groupby(["Gender","SN"])["Price"].sum()

#Create new data frame with Purchase totals for each user in each gender and calcualte average
gender_sn_groups_mean =pd.DataFrame(gender_sn_groups.groupby(["Gender"]).mean())

gender_sn_groups_mean2 = gender_sn_groups_mean.reset_index()
#gender_sn_groups_mean2

#Apply Formatting
purchase_stats = pd.options.display.float_format = '${:,.2f}'.format

# Merge data frames to display results
gender_analysis_df = gender_purchases_df.merge(gender_sn_groups_mean2, on="Gender", how="left")
gender_analysis_df

#Rename columns
gender_analysis_df2 = gender_analysis_df.rename(columns={"Price":"Avg_Total_Purchase_Per_Person"})


gender_analysis_df2



,Gender,Purchase_Count,Total_Purchase_Value,Average_Purchase_Price,Avg_Total_Purchase_Per_Person
0,Female,113,$361.94,$3.20,$4.47
1,Male,652,"$1,967.64",$3.02,$4.07
2,Other / Non-Disclosed,15,$50.19,$3.35,$4.56


In [54]:
# Age Demographics
#----------------------------------

# Find lowest and highest age to set bins

purchase_data_nodups_df["Age"].max()
purchase_data_nodups_df["Age"].min()

# Define Bins Values and Names

bins= [0,9,14,19,24,29,34,39,100]
bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

# Create new column in dataframe with age group
purchase_data_nodups_df["Age_Group"] = pd.cut(purchase_data_nodups_df["Age"], bins, labels=bin_names)
#purchase_data_nodups_df

#Group by Age Group and calculate total purchases, total value, and avg purchage for each age group
age_groups1 = purchase_data_nodups_df.groupby(["Age_Group"])["SN"].count()

#Convert Series to Data Frame
age_groups1_df = pd.DataFrame(age_groups1)
#age_groups1_df

#Format Dataframe
age_groups = age_groups1_df.reset_index().rename(columns={"SN":"Total_Count"})
#age_groups

# Calculate Precentage of Player by Age Group
age_groups["Percentage_of_Players"] = age_groups["Total_Count"] / total_players*100

#Formate Values
age_groups["Percentage_of_Players"] = age_groups["Percentage_of_Players"].map('{:,.2f}%'.format)


age_groups


,Age_Group,Total_Count,Percentage_of_Players
0,<10,17,2.95%
1,10-14,22,3.82%
2,15-19,107,18.58%
3,20-24,258,44.79%
4,25-29,77,13.37%
5,30-34,52,9.03%
6,35-39,31,5.38%
7,40+,12,2.08%


In [62]:
# Purchase Analysis by Age Group

#Define bins

bins= [0,9,14,19,24,29,34,39,100]

bin_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#Create new column in dataframe with age group
purchase_data["Age_Group"] = pd.cut(purchase_data["Age"], bins, labels=bin_names)
#purchase_data

#Group by Age Group and Calculate total purchase count and average
age_groups3 = purchase_data.groupby(["Age_Group"]).agg({"Purchase ID":"count", "Price":"mean"})
#age_groups3

#Group by Age Group and Calculate total purchase value
age_groups4 = purchase_data.groupby(["Age_Group"]).agg({"Price":"sum"})
#age_groups4

#Group by Age Group and SN and Calculate total purchase value per player
age_groups5 = purchase_data.groupby(["Age_Group","SN"])["Price"].agg({"Price":"sum"})

#Group by Age Group and SN and Calculate average purchase value per player
age_groups6 =age_groups5.groupby(["Age_Group"]).mean()
age_groups6

# merge data frame on age group to display results
age_analysis_df = age_groups3.merge(age_groups4,on=["Age_Group"],how="left").merge(age_groups6,on=["Age_Group"],how="left")


#Format data frame
age_analysis_df = age_analysis_df.reset_index().rename(columns={"Purchase ID":"Purchase_Count", 
                                                            "Price_x":"Average_Purchase_Price",
                                                            "Price_y":"Total_Purchase_Value",
                                                            "Price":"Avg_Total_Purchase_per_Person"})

age_analysis_df

C:\Users\jprom\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version


,Age_Group,Purchase_Count,Average_Purchase_Price,Total_Purchase_Value,Avg_Total_Purchase_per_Person
0,<10,23,$3.35,$77.13,$4.54
1,10-14,28,$2.96,$82.78,$3.76
2,15-19,136,$3.04,$412.89,$3.86
3,20-24,365,$3.05,"$1,114.06",$4.32
4,25-29,101,$2.90,$293.00,$3.81
5,30-34,73,$2.93,$214.00,$4.12
6,35-39,41,$3.60,$147.67,$4.76
7,40+,13,$2.94,$38.24,$3.19


In [63]:
#Top Spenders
#----------------------

#Group purchase data by SN and calculate total number of purchases and average purchase price
sn_groups1 = purchase_data.groupby(["SN"]).agg({"Purchase ID":"count", "Price":"mean"})
#sn_groups1

#Group by player and Calculate total purchase value for each player
sn_groups2 = purchase_data.groupby(["SN"]).agg({"Price":"sum"})
#sn_groups2

#Merge two resulting dataframes to display results
top_spenders = sn_groups1.merge(sn_groups2,on=["SN"],how="left")
#top_spenders

# Format and Sort dataframe  in descending

top_spenders = top_spenders.rename(columns={"Purchase ID":"Purchase_Count", 
                                            "Price_x":"Avg_Purchase_Price",
                                            "Price_y":"Total_Purchase_Value"})

top_spenders = top_spenders.sort_values("Total_Purchase_Value", ascending=False)

#Display Top 5 spenders per Total Purchase Value

top_spenders.head(5)

,Purchase_Count,Avg_Purchase_Price,Total_Purchase_Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [78]:
# Most Popular Items 
#------------------------

#Groupby Item ID and calculate total purchase count and value for each item
item_groups1 = purchase_data.groupby(["Item ID"]).agg({"Purchase ID":"count", "Price":"sum"})

#Format dataframe
item_groups1 = item_groups1.reset_index().rename(columns={"Item ID":"Item_ID"})

#Create new data frame to only include Item Name and Price
item_groups2=pd.DataFrame({"Item_ID":purchase_data["Item ID"],
                           "Item_Name":purchase_data["Item Name"],
                           "Purchase_Price":purchase_data["Price"]})

#Drop duplicate from Date Frame to obtain list of unique items
item_groups2= pd.DataFrame(item_groups2.drop_duplicates(subset=["Item_ID"]))
#len(item_groups2)
#item_groups2

#Merge results data frame and list of unique items to display results
top_items = item_groups2.merge(item_groups1 ,on=["Item_ID"],how="left")
#len(top_items)
#top_items

# Format Data Frame
top_items = top_items.rename(columns={"Purchase ID":"Purchase_Count","Price":"Total_Purchase_Value"})
#top_items

#Sort Data Frame to Display top selling items
top_items = top_items.sort_values("Purchase_Count", ascending=False)

#Display top 5 selling items
top_items.head(5)



,Item_ID,Item_Name,Purchase_Price,Purchase_Count,Total_Purchase_Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
0,108,"Extraction, Quickblade Of Trembling Hands",$3.53,9,$31.77
17,82,Nirvana,$4.90,9,$44.10
94,145,Fiery Glass Crusader,$4.58,9,$41.22
134,19,"Pursuit, Cudgel of Necromancy",$1.02,8,$8.16


In [75]:
# Most Profitable items

#Sort top items Dataframe by Purchase value
top_items_value = top_items.sort_values("Total_Purchase_Value", ascending=False)

# Display Top 5 most profitable itmems
top_items_value.head(5)

,Item_ID,Item_Name,Purchase_Price,Purchase_Count,Total_Purchase_Value
24,178,"Oathbreaker, Last Hope of the Breaking Storm",$4.23,12,$50.76
17,82,Nirvana,$4.90,9,$44.10
94,145,Fiery Glass Crusader,$4.58,9,$41.22
2,92,Final Critic,$4.88,8,$39.04
112,103,Singed Scalpel,$4.35,8,$34.80
